### Este notebook ayuda a visualizar y modularizar el proceso de preprocesamiento de los datos

In [3]:
# Módulo con los import necesarios para el proyecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# Cargar el archivo CSV en un DataFrame de pandas
results_df = pd.read_csv('../data/results.csv')

# Mostrar las primeras filas del DataFrame
results_df.head()

,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup
0,1871-03-27,Scotland,England,1,0,1871 Scotland v England International,Raeburn Place,Edinburgh,Scotland,False,False
1,1872-02-05,England,Scotland,2,1,1871-72 Home Nations International,The Oval,London,England,False,False
2,1873-03-03,Scotland,England,0,0,1872-73 Home Nations International,West of Scotland F.C.,Glasgow,Scotland,False,False
3,1874-02-23,England,Scotland,1,0,1873-74 Home Nations International,The Oval,London,England,False,False
4,1875-02-15,England,Ireland,2,0,1874-75 Home Nations rugby union matches,The Oval,London,England,False,False


In [5]:
# Mostrar información general del dataframe
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         2783 non-null   object
 1   home_team    2783 non-null   object
 2   away_team    2783 non-null   object
 3   home_score   2783 non-null   int64 
 4   away_score   2783 non-null   int64 
 5   competition  2760 non-null   object
 6   stadium      2783 non-null   object
 7   city         2783 non-null   object
 8   country      2783 non-null   object
 9   neutral      2783 non-null   bool  
 10  world_cup    2783 non-null   bool  
dtypes: bool(2), int64(2), object(7)
memory usage: 201.2+ KB
